In [4]:
import pandas as pd
file_path = "C:\\Users\\xinyi\\Desktop\\recommend_trx_simple\\train.csv"
df = pd.read_csv(file_path)

In [2]:
!pip install xgboost

     ------------------------------------ 124.9/124.9 MB 370.6 kB/s eta 0:00:00


In [5]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score

# 2. 分离特征和标签
X = df.drop(columns=['reqId', 'label']) # 删除 id 和 label 列
y = df['label'] # 标签列

# 3. 检查数据形状
print("Features shape:", X.shape) # (180000, 200) 假设有 180,000 行和 200 个特征
print("Labels shape:", y.shape) # (180000,) 标签列有 180,000 行

Features shape: (180000, 200)
Labels shape: (180000,)


In [7]:
# 2. 数据预处理
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) # 对特征进行标准化

# 3. 划分训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 4. 定义并训练 XGBoost 模型
model = xgb.XGBClassifier(scale_pos_weight=3) # 调整权重来处理类别不平衡
model.fit(X_train, y_train)

# 5. 预测并评估
y_pred = model.predict(X_val)
y_pred_proba = model.predict_proba(X_val)[:, 1] # 获取概率值

# 评估性能
print("Accuracy:", accuracy_score(y_val, y_pred))
print("AUC:", roc_auc_score(y_val, y_pred_proba))

Accuracy: 0.90875
AUC: 0.8605609794593789


In [9]:
print(y_pred_proba)

[0.04500609 0.11988395 0.08563881 ... 0.03675099 0.01548816 0.02969294]


In [12]:
model.fit(X_scaled, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [11]:
test_df = pd.read_csv('C:\\Users\\xinyi\\Desktop\\recommend_trx_simple\\test.feature.csv')

In [13]:
test_X = test_df.drop(columns=['reqId'])
test_X_scaled = scaler.transform(test_X)# 删除 id 列

In [14]:
test_pred_proba = model.predict_proba(test_X_scaled)[:, 1] # 获取概率值
print(test_pred_proba)

[0.02106927 0.28977653 0.01447096 ... 0.02321903 0.20744029 0.70445734]


In [16]:
import os

# 1. 创建一个新的 DataFrame，包含预测结果
result_df = pd.DataFrame({'label': test_pred_proba})
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop', 'recommend_trx_simple.csv')

# 保存到桌面
result_df.to_csv(desktop_path, index=False)